# Deep Neural Network Model Design

**By: M. Alwi Sukra**

In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

In [3]:
os.environ["TF_ENABLE_CUBLAS_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_CUDNN_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_CUDNN_RNN_TENSOR_OP_MATH_FP32"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_LOSS_SCALING"] = "1"

## 1. LSTM With Base Feature

In [4]:
DATA_PATH = "datasets/lstm-base"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "LSTM_base-features"

# get data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

# model structure
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(1, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [5]:
MAX_EPOCHS = 100
model.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100
262/262 - 4s - loss: 0.7035 - binary_accuracy: 0.5054 - val_loss: 0.6979 - val_binary_accuracy: 0.5059 - lr: 0.0010
Epoch 2/100
262/262 - 4s - loss: 0.6965 - binary_accuracy: 0.4685 - val_loss: 0.6957 - val_binary_accuracy: 0.4424 - lr: 0.0010
Epoch 3/100
262/262 - 4s - loss: 0.6949 - binary_accuracy: 0.4381 - val_loss: 0.6944 - val_binary_accuracy: 0.4822 - lr: 0.0010
Epoch 4/100
262/262 - 4s - loss: 0.6938 - binary_accuracy: 0.4870 - val_loss: 0.6934 - val_binary_accuracy: 0.5005 - lr: 0.0010
Epoch 5/100
262/262 - 5s - loss: 0.6930 - binary_accuracy: 0.5050 - val_loss: 0.6928 - val_binary_accuracy: 0.5414 - lr: 0.0010
Epoch 6/100
262/262 - 4s - loss: 0.6923 - binary_accuracy: 0.5853 - val_loss: 0.6918 - val_binary_accuracy: 0.5436 - lr: 0.0010
Epoch 7/100
262/262 - 4s - loss: 0.6911 - binary_accuracy: 0.5418 - val_loss: 0.6899 - val_binary_accuracy: 0.6416 - lr: 0.0010
Epoch 8/100
262/262 - 4s - loss: 0.6816 - binary_accuracy: 0.6317 - val_loss: 0.6648 - val_binary_accura

In [6]:
model.save("saved-models/"+model_name)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved-models/LSTM_base-features\assets


## 1. LSTM With Aggregate Feature

In [7]:
DATA_PATH = "datasets/lstm-agg"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "LSTM_agg-features"

# get data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

# model structure
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.LSTM(1, input_shape=(X_train.shape[1], X_train.shape[2])))
model2.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model2.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [8]:
MAX_EPOCHS = 100
model2.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100
262/262 - 4s - loss: 0.6804 - binary_accuracy: 0.5138 - val_loss: 0.6577 - val_binary_accuracy: 0.4833 - lr: 0.0010
Epoch 2/100
262/262 - 4s - loss: 0.6010 - binary_accuracy: 0.7185 - val_loss: 0.5340 - val_binary_accuracy: 0.7729 - lr: 0.0010
Epoch 3/100
262/262 - 4s - loss: 0.4771 - binary_accuracy: 0.8208 - val_loss: 0.4633 - val_binary_accuracy: 0.8364 - lr: 0.0010
Epoch 4/100
262/262 - 4s - loss: 0.4591 - binary_accuracy: 0.8276 - val_loss: 0.4487 - val_binary_accuracy: 0.8375 - lr: 0.0010
Epoch 5/100
262/262 - 4s - loss: 0.4486 - binary_accuracy: 0.8294 - val_loss: 0.4343 - val_binary_accuracy: 0.8396 - lr: 0.0010
Epoch 6/100
262/262 - 4s - loss: 0.4148 - binary_accuracy: 0.8499 - val_loss: 0.3951 - val_binary_accuracy: 0.8654 - lr: 0.0010
Epoch 7/100
262/262 - 4s - loss: 0.4037 - binary_accuracy: 0.8536 - val_loss: 0.3898 - val_binary_accuracy: 0.8654 - lr: 0.0010
Epoch 8/100
262/262 - 4s - loss: 0.4006 - binary_accuracy: 0.8531 - val_loss: 0.3867 - val_binary_accura

In [9]:
model2.save("saved-models/"+model_name)

INFO:tensorflow:Assets written to: saved-models/LSTM_agg-features\assets


## 1. LSTM With Aggregate Feature

In [10]:
DATA_PATH = "datasets/lstm-all"

# get X and y
X = np.load(f"{DATA_PATH}-X.npy")
y = np.load(f"{DATA_PATH}-y.npy")

model_name = "LSTM_all-features"

# get data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
np.save("datasets/X_test_"+model_name, X_test)
np.save("datasets/y_test_"+model_name, y_test)

# model structure
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.LSTM(1, input_shape=(X_train.shape[1], X_train.shape[2])))
model3.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

# compile moodel
opt = tf.keras.optimizers.Adam()
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model3.compile(optimizer=opt, loss="binary_crossentropy", metrics = [acc])

# setup callbacks
checkpoint_filepath = f"saved-models/{model_name}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{model_name}.csv"
# callbacks
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)
csv_log = CSVLogger(log_filepath, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks = [checkpoint, csv_log, reduce_lr, early_stop]

In [11]:
MAX_EPOCHS = 100
model3.fit(X_train, y_train, validation_split=0.1, epochs=MAX_EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/100
262/262 - 4s - loss: 0.6950 - binary_accuracy: 0.4555 - val_loss: 0.6937 - val_binary_accuracy: 0.4941 - lr: 0.0010
Epoch 2/100
262/262 - 4s - loss: 0.6932 - binary_accuracy: 0.5125 - val_loss: 0.6939 - val_binary_accuracy: 0.4941 - lr: 0.0010
Epoch 3/100
262/262 - 4s - loss: 0.6923 - binary_accuracy: 0.5126 - val_loss: 0.6901 - val_binary_accuracy: 0.4941 - lr: 0.0010
Epoch 4/100
262/262 - 4s - loss: 0.6374 - binary_accuracy: 0.7496 - val_loss: 0.6030 - val_binary_accuracy: 0.7503 - lr: 0.0010
Epoch 5/100
262/262 - 4s - loss: 0.5470 - binary_accuracy: 0.8001 - val_loss: 0.5225 - val_binary_accuracy: 0.7987 - lr: 0.0010
Epoch 6/100
262/262 - 4s - loss: 0.4831 - binary_accuracy: 0.8294 - val_loss: 0.4664 - val_binary_accuracy: 0.8310 - lr: 0.0010
Epoch 7/100
262/262 - 4s - loss: 0.4409 - binary_accuracy: 0.8496 - val_loss: 0.4416 - val_binary_accuracy: 0.8418 - lr: 0.0010
Epoch 8/100
262/262 - 4s - loss: 0.4194 - binary_accuracy: 0.8578 - val_loss: 0.4257 - val_binary_accura

In [12]:
model3.save("saved-models/"+model_name)

INFO:tensorflow:Assets written to: saved-models/LSTM_all-features\assets
